In [81]:
from glob import glob
import os
import re

In [80]:
folders = [f for f in sorted(glob('*')) if os.path.isdir(f)]
folders.remove('LEGOv2')

print(len(folders))
for folder in sorted(folders, key=lambda x: len(x)):
    print(folder)
    
emotions = ['anger', 'disgust', 
            'fear', 'happiness', 
            'sadness', 'surprise', 'neutral']

print()

print(emotions)

13
ANAD
EEKK
TESS
BAVED
SAVEE
ShEMO
VIVAE
CREMA-D
RAVDESS
CaFE_48k
JL corpus
URDU-Dataset
Acted Emotional Speech Dynamic Database

['anger', 'disgust', 'fear', 'happiness', 'sadness', 'surprise', 'neutral']


In [70]:
import csv

anad_files = sorted(glob('ANAD/*/*/*.wav'))
print(f'ANAD FILES {len(anad_files)}')
print(anad_files[0])
print()

file_emotion_dict = dict()

with open('./ANAD/ANAD.csv', 'r') as f:
    reader = csv.reader(f)
    for i, line in enumerate(reader):
        if i == 0:
            continue
        f, emotion = line[0].strip("'"), line[1]
        file_emotion_dict[f] = emotion

# from collections import Counter

missing_file_list = list()
missing = 0
found = 0

print(len(file_emotion_dict))

for f in anad_files:
    f_name = os.path.basename(f) 
    if f_name not in file_emotion_dict:
        missing += 1
        missing_file_list.append(f_name)
#         print(f_name)
    else:
        found += 1
        
print(f'MISSING {missing:4}')
print(f'FOUND   {found:>4}')

# files = [os.path.basename(f) for f in anad_files]

# for key in file_emotion_dict:
#     if key not in files:
#         print(key)

# c = Counter([os.path.dirname(f) for f in anad_files])
# for e in c.items():
#     print(e)

ANAD FILES 1420
ANAD/1sec_segmented_part1/1sec_segmented_part1/V1_1 (1).wav

1383
MISSING   86
FOUND   1334


In [97]:
def get_savee_meta(file):
    speaker = 'SAVEE_' + file.split('/')[-2]
    pattern_emotion = [('a\d+.wav', 'anger'),
                       ('d\d+.wav', 'disgust'),
                       ('f\d+.wav', 'fear'),
                       ('h\d+.wav', 'happiness'),
                       ('n\d+.wav', 'neutral'),
                       ('sa\d+.wav', 'sadness'),
                       ('su\d+.wav', 'surprise'),
                      ]

    f_name = os.path.basename(file)
    for pattern, emotion in pattern_emotion:
        if re.match(pattern, f_name):
            e = emotion
            break
    return file, speaker, e

savee_meta = list(map(get_savee_meta, savee_files))

print(savee_meta[0])

('SAVEE/AudioData/DC/a01.wav', 'SAVEE_DC', 'anger')


In [ ]:
def get_tess_meta(file):
    
    return

tess_files

In [74]:
# https://github.com/SuperKogito/SER-datasets

# SAVEE
# English (British)
# SAVEE FILES 480
# SAVEE/AudioData/DC/a01.wav
# 480 British English utterances by 4 males actors.
# 7 emotions: anger, disgust, fear, happiness, sadness, surprise and neutral.
savee_files = sorted(glob('SAVEE/AudioData/*/*.wav'))
print(f'SAVEE FILES {len(savee_files)}')
print(savee_files[0])
print()

# TESS
# English
# TESS FILES 2800
# TESS/OAF_back_angry.wav
# 2800 recording by 2 actresses.
# 7 emotions: anger, disgust, fear, happiness, pleasant surprise, sadness, and neutral.
tess_files = sorted(glob('TESS/*.wav'))
print(f'TESS FILES {len(tess_files)}')
print(tess_files[0])
print()

# RAVDESS
# English
# 7356 recordings by 24 actors.
# Speech file (Audio_Speech_Actors_01-24.zip, 215 MB) contains 1440 files: 
# 60 trials per actor x 24 actors = 1440.
# Song file (Audio_Song_Actors_01-24.zip, 198 MB) contains 1012 files: 
# 44 trials per actor x 23 actors = 1012.
# RAVDESS FILES 1440
# RAVDESS/Actor_01/03-01-01-01-01-01-01.wav
# 7 emotions: calm, happy, sad, angry, fearful, surprise, and disgust
ravdess_files = sorted(glob('RAVDESS/*/*.wav'))
print(f'RAVDESS FILES {len(ravdess_files)}')
print(ravdess_files[0])
print()

# CREMA-D
# English
# 7442 clip of 12 sentences spoken by 91 actors (48 males and 43 females).
# CREAMA-D FILES 7442
# CREMA-D/AudioWAV/1001_DFA_ANG_XX.wav
# 6 emotions: angry, disgusted, fearful, happy, neutral, and sad
crema_d_files = sorted(glob('CREMA-D/AudioWAV/*.wav'))
print(f'CREAMA-D FILES {len(crema_d_files)}')
print(crema_d_files[0])
print()

# URDU
# Urdu
# 400 utterances by 38 speakers (27 male and 11 female).
# URDU FILES 400
# URDU-Dataset/Angry/SM1_F10_A010.wav
# 4 emotions: angry, happy, neutral, and sad.
urdu_files = sorted(glob('URDU-Dataset/*/*.wav'))
print(f'URDU FILES {len(urdu_files)}')
print(urdu_files[0])
print()

# BAVED
# Arabic
# 1935 recording by 61 speakers (45 male and 16 female).
# BAVED FILES 1935
# BAVED/0/0-m-21-0-1-105.wav
baved_files = sorted(glob('BAVED/*/*.wav'))
print(f'BAVED FILES {len(baved_files)}')
print(baved_files[0])
print()

# VIVAE
# non-speech, 1085 audio file by ~12 speakers.
# VIVAE FILES 1085
# VIVAE/full_set/S01_achievement_low_01.wav
# non-speech 6 emotions: achievement, anger, fear, pain, pleasure, and surprise 
# with 3 emotional intensities (low, moderate, strong, peak).
vivae_files = sorted(glob('VIVAE/full_set/*.wav'))
print(f'VIVAE FILES {len(vivae_files)}')
print(vivae_files[0])
print()

# ShEMO
# 3000 semi-natural utterances, equivalent to 3 hours and 25 minutes 
# of speech data from online radio plays by 87 native-Persian speakers.
# ShEMO FILES 3000
# ShEMO/female/F01A01.wav
# 6 emotions: anger, fear, happiness, sadness, neutral and surprise.
shemo_files = sorted(glob('ShEMO/*/*.wav'))
print(f'ShEMO FILES {len(shemo_files)}')
print(shemo_files[0])
print()

# JL corpus
# 2400 recording of 240 sentences by 4 actors (2 males and 2 females).
# JL corpus FILES 2400
# ShEMO/female/F01A01.wav
# 5 primary emotions: angry, sad, neutral, happy, excited. 
# 5 secondary emotions: anxious, apologetic, pensive, worried, enthusiastic.
jl_corpus_files = sorted(glob('JL corpus/Raw JL corpus (unchecked and unannotated)/JL(wav+txt)/*.wav'))
print(f'JL corpus FILES {len(jl_corpus_files)}')
print(shemo_files[0])
print()

# CaFE
# French (Canadian)
# 6 different sentences by 12 speakers (6 fmelaes + 6 males).
# 12 * 6 * (6 * 2 + 1) 
# CaFE FILES 936
# CaFE_48k/ColŠre/Faible/01-C-1-1.wav
# 7 emotions: happy, sad, angry, fearful, surprise, disgust and neutral. 
# Each emotion is acted in 2 different intensities.
cafe_files = sorted(glob('CaFE_48k/*/*/*.wav') + glob('CaFE_48k/*/*.wav'))
print(f'CaFE FILES {len(cafe_files)}')
print(cafe_files[0])
print()

# ANAD
# Arabic
# 1384 recording by multiple speakers.
# ANAD FILES 1420
# ANAD/1sec_segmented_part1/1sec_segmented_part1/V1_1 (1).wav
# 3 emotions: angry, happy, surprised.
anad_files = sorted(glob('ANAD/*/*/*.wav'))
print(f'ANAD FILES {len(anad_files)}')
print(anad_files[0])
print()

# EEKK
# Estonian
# 26 text passage read by 10 speakers.
# EEKK FILES 1164
# EEKK/ekorpus/105.wav
# 4 main emotions: joy, sadness, anger and neutral.
eekk_files = sorted(glob('EEKK/ekorpus/*.wav'))
print(f'EEKK FILES {len(eekk_files)}')
print(eekk_files[0])
print()

# AESDD
# Greek
# around 500 utterances by a diverse group of actors (over 5 actors) simlating various emotions.
# AESDD FILES 605
# Acted Emotional Speech Dynamic Database/anger/a01 (1).wav
# 5 emotions: anger, disgust, fear, happiness, and sadness.
aesdd_files = sorted(glob('Acted Emotional Speech Dynamic Database/*/*.wav'))
print(f'AESDD FILES {len(aesdd_files)}')
print(aesdd_files[0])
print()




SAVEE FILES 480
SAVEE/AudioData/DC/a01.wav

TESS FILES 2800
TESS/OAF_back_angry.wav

RAVDESS FILES 1440
RAVDESS/Actor_01/03-01-01-01-01-01-01.wav

CREAMA-D FILES 7442
CREMA-D/AudioWAV/1001_DFA_ANG_XX.wav

URDU FILES 400
URDU-Dataset/Angry/SM1_F10_A010.wav

BAVED FILES 1935
BAVED/0/0-m-21-0-1-105.wav

VIVAE FILES 1085
VIVAE/full_set/S01_achievement_low_01.wav

ShEMO FILES 3000
ShEMO/female/F01A01.wav

JL corpus FILES 2400
ShEMO/female/F01A01.wav

CaFE FILES 936
CaFE_48k/ColŠre/Faible/01-C-1-1.wav

ANAD FILES 1420
ANAD/1sec_segmented_part1/1sec_segmented_part1/V1_1 (1).wav

EEKK FILES 1164
EEKK/ekorpus/105.wav

AESDD FILES 605
Acted Emotional Speech Dynamic Database/anger/a01 (1).wav



In [ ]:
EMODB
IEMCAO
